In [2]:
import pandas as pd
import numpy as np
import recordlinkage as rl
import re

# POKEMON

In [2]:
dex_scrap = pd.read_json('Scraping_datasets/pokemondb/pokedex.json')
dex_scrap.tail()

,#,Name,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,sprite_url,Variant
1070,902,Basculegion,Water,Ghost,530,120,92,65,100,75,78,https://img.pokemondb.net/s.png,Female
1071,903,Sneasler,Poison,Fighting,510,80,130,60,40,80,120,https://img.pokemondb.net/s.png,NaN
1072,904,Overqwil,Dark,Poison,510,85,115,95,65,65,85,https://img.pokemondb.net/s.png,NaN
1073,905,Enamorus,Fairy,Flying,580,74,115,70,135,80,106,https://img.pokemondb.net/s.png,Incarnate Forme
1074,905,Enamorus,Fairy,Flying,580,74,115,110,135,100,46,https://img.pokemondb.net/s.png,Therian Forme


In [3]:
species = pd.read_json('API_per_integrazione/species.json.gz', compression='gzip', orient='records', lines=True)
species.tail()

,ID,Name,Varieties,Generation,Evolves_from,Has_gender_diff,Rarity
893,894,Regieleki,"[{'is_default': True, 'pokemon': {'name': 'reg...",8,None,False,Legendary
894,895,Regidrago,"[{'is_default': True, 'pokemon': {'name': 'reg...",8,None,False,Legendary
895,896,Glastrier,"[{'is_default': True, 'pokemon': {'name': 'gla...",8,None,False,Legendary
896,897,Spectrier,"[{'is_default': True, 'pokemon': {'name': 'spe...",8,None,False,Legendary
897,898,Calyrex,"[{'is_default': True, 'pokemon': {'name': 'cal...",8,None,False,Legendary


In [4]:
#Inizializzo l'indexer per l'integrazione
indexer = rl.Index()
indexer.full()

<Index>

In [5]:
#Valuto il numero di coppie candidate al linking
candidates = indexer.index(species, dex_scrap)
print(len(candidates))

965350


In [6]:
compare = rl.Compare()
compare.string('Name', 'Name', threshold=0.85, label='Name')
compare.exact('ID', '#', label='ID')

features = compare.compute(candidates, species, dex_scrap)

In [7]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0      1054
1.0        23
0.0    964273
dtype: int64

Abbiamo 1054 match completi (nome+ID) e 23 match singoli (o nome o ID)

In [8]:
potential_matches = features[features.sum(axis=1) > 0].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'Name':'ID'].sum(axis=1)
potential_matches

,level_0,level_1,Name,ID,Score
0,0,0,1.0,1,2.0
1,1,1,1.0,1,2.0
2,2,2,1.0,1,2.0
3,2,3,1.0,1,2.0
4,3,4,1.0,1,2.0
...,...,...,...,...,...
1072,895,1061,1.0,1,2.0
1073,896,1062,1.0,1,2.0
1074,897,1063,1.0,1,2.0
1075,897,1064,1.0,1,2.0


In [9]:
potential_matches[potential_matches['Score'] == 1.0]

,level_0,level_1,Name,ID,Score
40,28,40,0.0,1,1.0
42,29,44,1.0,0,1.0
44,31,43,0.0,1,1.0
45,32,41,1.0,0,1.0
161,118,355,1.0,0,1.0
164,121,161,0.0,1,1.0
165,121,162,0.0,1,1.0
184,136,292,1.0,0,1.0
285,221,1022,1.0,0,1.0
297,232,180,1.0,0,1.0


Analizzo manualmente i match con score pari a 1 (probabilmente basterebbe alzare la soglia, ma controllo ugualmente):

In [10]:
#Match numero 1: Nidoran femmina ---> VA TENUTO
print(species.loc[28])
print(dex_scrap.loc[40])

ID                                                                29
Name                                                       Nidoran-f
Varieties          [{'is_default': True, 'pokemon': {'name': 'nid...
Generation                                                         1
Evolves_from                                                    None
Has_gender_diff                                                False
Rarity                                                        Common
Name: 28, dtype: object
#                                                            29
Name                                                   Nidoran♀
Type1                                                    Poison
Type2                                                       NaN
Total                                                       275
HP                                                           55
Attack                                                       47
Defense                                      

In [11]:
#Match numero 2: Nidorino con Nidorina ---> ELIMINATO
print(species.loc[29])
print(dex_scrap.loc[44])

ID                                                                30
Name                                                        Nidorina
Varieties          [{'is_default': True, 'pokemon': {'name': 'nid...
Generation                                                         1
Evolves_from                                               Nidoran-f
Has_gender_diff                                                False
Rarity                                                        Common
Name: 29, dtype: object
#                                                            33
Name                                                   Nidorino
Type1                                                    Poison
Type2                                                       NaN
Total                                                       365
HP                                                           61
Attack                                                       72
Defense                                      

In [12]:
#Match numero 3: Nidoran maschio ---> VA TENUTO
print(species.loc[31])
print(dex_scrap.loc[43])

ID                                                                32
Name                                                       Nidoran-m
Varieties          [{'is_default': True, 'pokemon': {'name': 'nid...
Generation                                                         1
Evolves_from                                                    None
Has_gender_diff                                                False
Rarity                                                        Common
Name: 31, dtype: object
#                                                            32
Name                                                   Nidoran♂
Type1                                                    Poison
Type2                                                       NaN
Total                                                       273
HP                                                           46
Attack                                                       57
Defense                                      

In [13]:
#Match numero 4: Nidorino con Nidorina ---> ELIMINATO
print(species.loc[32])
print(dex_scrap.loc[41])

ID                                                                33
Name                                                        Nidorino
Varieties          [{'is_default': True, 'pokemon': {'name': 'nid...
Generation                                                         1
Evolves_from                                               Nidoran-m
Has_gender_diff                                                False
Rarity                                                        Common
Name: 32, dtype: object
#                                                            30
Name                                                   Nidorina
Type1                                                    Poison
Type2                                                       NaN
Total                                                       365
HP                                                           70
Attack                                                       62
Defense                                      

In [14]:
#Match numero 5: Slaking e Seaking sono diversi ---> ELIMINATO
print(species.loc[118])
print(dex_scrap.loc[355])

ID                                                               119
Name                                                         Seaking
Varieties          [{'is_default': True, 'pokemon': {'name': 'sea...
Generation                                                         1
Evolves_from                                                 Goldeen
Has_gender_diff                                                 True
Rarity                                                        Common
Name: 118, dtype: object
#                                                           289
Name                                                    Slaking
Type1                                                    Normal
Type2                                                       NaN
Total                                                       670
HP                                                          150
Attack                                                      160
Defense                                     

In [15]:
#Match numero 6: Mr Mime ---> VA TENUTO
print(species.loc[121])
print(dex_scrap.loc[161])

ID                                                               122
Name                                                         Mr-mime
Varieties          [{'is_default': True, 'pokemon': {'name': 'mr-...
Generation                                                         1
Evolves_from                                                 Mime-jr
Has_gender_diff                                                False
Rarity                                                        Common
Name: 121, dtype: object
#                                                           122
Name                                                        Mr.
Type1                                                   Psychic
Type2                                                     Fairy
Total                                                       460
HP                                                           40
Attack                                                       45
Defense                                     

In [16]:
#Match numero 7: Mr Mime ---> VA TENUTO
print(species.loc[121])
print(dex_scrap.loc[162])

ID                                                               122
Name                                                         Mr-mime
Varieties          [{'is_default': True, 'pokemon': {'name': 'mr-...
Generation                                                         1
Evolves_from                                                 Mime-jr
Has_gender_diff                                                False
Rarity                                                        Common
Name: 121, dtype: object
#                                                           122
Name                                                        Mr.
Type1                                                       Ice
Type2                                                   Psychic
Total                                                       460
HP                                                           50
Attack                                                       65
Defense                                     

In [17]:
#Match numero 8: Porygon e Porygon2 sono diversi ---> ELIMINATO
print(species.loc[136])
print(dex_scrap.loc[292])

ID                                                               137
Name                                                         Porygon
Varieties          [{'is_default': True, 'pokemon': {'name': 'por...
Generation                                                         1
Evolves_from                                                    None
Has_gender_diff                                                False
Rarity                                                        Common
Name: 136, dtype: object
#                                                           233
Name                                                   Porygon2
Type1                                                    Normal
Type2                                                       NaN
Total                                                       515
HP                                                           85
Attack                                                       80
Defense                                     

In [18]:
#Match numero 9: Corsola e Cursola sono diversi ---> ELIMINATO
print(species.loc[221])
print(dex_scrap.loc[1022])

ID                                                               222
Name                                                         Corsola
Varieties          [{'is_default': True, 'pokemon': {'name': 'cor...
Generation                                                         2
Evolves_from                                                    None
Has_gender_diff                                                False
Rarity                                                        Common
Name: 221, dtype: object
#                                                           864
Name                                                    Cursola
Type1                                                     Ghost
Type2                                                       NaN
Total                                                       510
HP                                                           60
Attack                                                       95
Defense                                     

In [19]:
#Match numero 10: Porygon e Porygon2 sono diversi ---> ELIMINATO
print(species.loc[232])
print(dex_scrap.loc[180])

ID                                                               233
Name                                                        Porygon2
Varieties          [{'is_default': True, 'pokemon': {'name': 'por...
Generation                                                         2
Evolves_from                                                 Porygon
Has_gender_diff                                                False
Rarity                                                        Common
Name: 232, dtype: object
#                                                           137
Name                                                    Porygon
Type1                                                    Normal
Type2                                                       NaN
Total                                                       395
HP                                                           65
Attack                                                       60
Defense                                     

In [20]:
#Match numero 11: Slaking e Seaking sono diversi ---> ELIMINATO
print(species.loc[288])
print(dex_scrap.loc[158])

ID                                                               289
Name                                                         Slaking
Varieties          [{'is_default': True, 'pokemon': {'name': 'sla...
Generation                                                         3
Evolves_from                                                Vigoroth
Has_gender_diff                                                False
Rarity                                                        Common
Name: 288, dtype: object
#                                                           119
Name                                                    Seaking
Type1                                                     Water
Type2                                                       NaN
Total                                                       450
HP                                                           80
Attack                                                       92
Defense                                     

In [21]:
#Match numero 12: Mime Jr è uguale ---> VA TENUTO
print(species.loc[438])
print(dex_scrap.loc[534])

ID                                                               439
Name                                                         Mime-jr
Varieties          [{'is_default': True, 'pokemon': {'name': 'mim...
Generation                                                         4
Evolves_from                                                    None
Has_gender_diff                                                False
Rarity                                                          Baby
Name: 438, dtype: object
#                                                           439
Name                                                       Mime
Type1                                                   Psychic
Type2                                                     Fairy
Total                                                       310
HP                                                           20
Attack                                                       25
Defense                                     

In [22]:
#Match numero 13 e 14: Minccino e Cinccino sono diversi ---> ELIMINATO
print(species.loc[571])
print(dex_scrap.loc[693])

print(species.loc[572])
print(dex_scrap.loc[692])

ID                                                               572
Name                                                        Minccino
Varieties          [{'is_default': True, 'pokemon': {'name': 'min...
Generation                                                         5
Evolves_from                                                    None
Has_gender_diff                                                False
Rarity                                                        Common
Name: 571, dtype: object
#                                                           573
Name                                                   Cinccino
Type1                                                    Normal
Type2                                                       NaN
Total                                                       470
HP                                                           75
Attack                                                       95
Defense                                     

In [23]:
#Match numero 15: C'è solo l'accento sbagliato ---> VA TENUTO
print(species.loc[668])
print(dex_scrap.loc[799])

ID                                                               669
Name                                                         Flabebe
Varieties          [{'is_default': True, 'pokemon': {'name': 'fla...
Generation                                                         6
Evolves_from                                                    None
Has_gender_diff                                                False
Rarity                                                        Common
Name: 668, dtype: object
#                                                           669
Name                                                    Flabébé
Type1                                                     Fairy
Type2                                                       NaN
Total                                                       303
HP                                                           44
Attack                                                       38
Defense                                     

In [24]:
#Match numero 16: Type NULL e Type: sono uguali ---> VA TENUTO
print(species.loc[771])
print(dex_scrap.loc[925])

ID                                                               772
Name                                                       Type-null
Varieties          [{'is_default': True, 'pokemon': {'name': 'typ...
Generation                                                         7
Evolves_from                                                    None
Has_gender_diff                                                False
Rarity                                                        Common
Name: 771, dtype: object
#                                                           772
Name                                                      Type:
Type1                                                    Normal
Type2                                                       NaN
Total                                                       534
HP                                                           95
Attack                                                       95
Defense                                     

In [25]:
#Match numero 17, 18, 19 e 20: I Tapu sono uguali ---> TENERE
print(species.loc[784])
print(dex_scrap.loc[939])

print(species.loc[785])
print(dex_scrap.loc[940])

print(species.loc[786])
print(dex_scrap.loc[941])

print(species.loc[787])
print(dex_scrap.loc[942])

ID                                                               785
Name                                                       Tapu-koko
Varieties          [{'is_default': True, 'pokemon': {'name': 'tap...
Generation                                                         7
Evolves_from                                                    None
Has_gender_diff                                                False
Rarity                                                     Legendary
Name: 784, dtype: object
#                                                           785
Name                                                       Tapu
Type1                                                  Electric
Type2                                                     Fairy
Total                                                       570
HP                                                           70
Attack                                                      115
Defense                                     

In [26]:
#Match numero 21 e 22: Cursola e Corsola sono diversi ---> ELIMINARE
print(species.loc[863])
print(dex_scrap.loc[279])

print(species.loc[863])
print(dex_scrap.loc[280])

ID                                                               864
Name                                                         Cursola
Varieties          [{'is_default': True, 'pokemon': {'name': 'cur...
Generation                                                         8
Evolves_from                                                 Corsola
Has_gender_diff                                                False
Rarity                                                        Common
Name: 863, dtype: object
#                                                           222
Name                                                    Corsola
Type1                                                     Water
Type2                                                      Rock
Total                                                       410
HP                                                           65
Attack                                                       55
Defense                                     

In [27]:
#Match numero 21 e 22: Sono lo stesso ---> TENERE
print(species.loc[865])
print(dex_scrap.loc[1024])

ID                                                               866
Name                                                         Mr-rime
Varieties          [{'is_default': True, 'pokemon': {'name': 'mr-...
Generation                                                         8
Evolves_from                                                 Mr-mime
Has_gender_diff                                                False
Rarity                                                        Common
Name: 865, dtype: object
#                                                           866
Name                                                        Mr.
Type1                                                       Ice
Type2                                                   Psychic
Total                                                       520
HP                                                           80
Attack                                                       85
Defense                                     

Si nota come nei match dove lo score è 1 (quindi non coincide il nome o non coincide l'ID) sono da scartare i match dove coincide il nome ma non l'ID (la coincidenza del nome viene dal fatto che sono nomi simili, dove magari cambia solo una lettera come nel caso "SLAKING-SEAKING") mentre sono da tenere i match dove non c'è coincidenza nel nome ma c'è coincidenza nell'ID. Quindi basta droppare queste righe:

In [28]:
potential_matches.drop(potential_matches[potential_matches.ID == 0].index, inplace=True)

In [29]:
potential_matches.head()

,level_0,level_1,Name,ID,Score
0,0,0,1.0,1,2.0
1,1,1,1.0,1,2.0
2,2,2,1.0,1,2.0
3,2,3,1.0,1,2.0
4,3,4,1.0,1,2.0


In [30]:
species.head()

,ID,Name,Varieties,Generation,Evolves_from,Has_gender_diff,Rarity
0,1,Bulbasaur,"[{'is_default': True, 'pokemon': {'name': 'bul...",1,None,False,Common
1,2,Ivysaur,"[{'is_default': True, 'pokemon': {'name': 'ivy...",1,Bulbasaur,False,Common
2,3,Venusaur,"[{'is_default': True, 'pokemon': {'name': 'ven...",1,Ivysaur,True,Common
3,4,Charmander,"[{'is_default': True, 'pokemon': {'name': 'cha...",1,None,False,Common
4,5,Charmeleon,"[{'is_default': True, 'pokemon': {'name': 'cha...",1,Charmander,False,Common


Procedo con il merging:

In [31]:
#Aggiungo le informazioni da species (la colonna riferita a species è level_0)
pokemon_complete = pd.merge(potential_matches, species, left_on="level_0", right_index=True)

In [32]:
#Aggiungo le informazioni da scraping (la colonna riferita a dex_scrap è level_1)
pokemon_complete = pd.merge(pokemon_complete, dex_scrap, left_on="level_1", right_index=True)

In [33]:
pokemon_complete.drop(["level_0", "level_1", "Name_x", "ID_x", "Score", "ID_y", "Name_y"], axis = 1, inplace=True)

In [34]:
pokemon_complete = pokemon_complete.reindex(columns=['#', 'Name', 'Generation', 'Rarity', 'Evolves_from', 'Has_gender_diff',
                                                    'Type1', 'Type2', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def',
                                                    'Speed', 'Variant', 'Varieties', 'sprite_url'])
pokemon_complete.reset_index(inplace=True)

In [35]:
pokemon_complete

,index,#,Name,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Variant,Varieties,sprite_url
0,0,1,Bulbasaur,1,Common,None,False,Grass,Poison,318,45,49,49,65,65,45,NaN,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,1,2,Ivysaur,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,NaN,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,2,3,Venusaur,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,NaN,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,3,Venusaur,1,Common,Ivysaur,True,Grass,Poison,625,80,100,123,122,120,80,Mega Venusaur,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,4,4,Charmander,1,Common,None,False,Fire,NaN,309,39,52,43,60,50,65,NaN,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,1072,896,Glastrier,8,Legendary,None,False,Ice,NaN,580,100,145,130,65,110,30,NaN,"[{'is_default': True, 'pokemon': {'name': 'gla...",https://img.pokemondb.net/sprites/sword-shield...
1062,1073,897,Spectrier,8,Legendary,None,False,Ghost,NaN,580,100,65,60,145,80,130,NaN,"[{'is_default': True, 'pokemon': {'name': 'spe...",https://img.pokemondb.net/sprites/sword-shield...
1063,1074,898,Calyrex,8,Legendary,None,False,Psychic,Grass,500,100,80,80,80,80,80,NaN,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...
1064,1075,898,Calyrex,8,Legendary,None,False,Psychic,Ice,680,100,165,150,85,130,50,Ice Rider,"[{'is_default': True, 'pokemon': {'name': 'cal...",https://img.pokemondb.net/sprites/sword-shield...


# ITEMS

In [3]:
item_scrap = pd.read_json('Scraping_datasets/pokemondb/items.json')
item_scrap.head()

,Name,Category,Effect,sprite_url
0,Ability Capsule,Hold items,A capsule that allows a Pokémon with two Abili...,https://img.pokemondb.net/sprites/items/abilit...
1,Ability Patch,General items,Changes a Pokémon's ability to its Hidden Abil...,https://img.pokemondb.net/s.png
2,Ability Urge,Battle items,"When used, it activates the Ability of an ally...",https://img.pokemondb.net/sprites/items/abilit...
3,Abomasite,Hold items,Enables Abomasnow to Mega Evolve during battle.,https://img.pokemondb.net/sprites/items/abomas...
4,Absolite,Hold items,Enables Absol to Mega Evolve during battle.,https://img.pokemondb.net/sprites/items/absoli...


In [7]:
item_api = pd.read_json('API_per_integrazione/items.json.gz', compression='gzip', orient='records', lines=True)
item_api.head()

,ID,Name,Attributes,Category,Effect
0,1,Master-ball,"[countable, consumable, usable-in-battle, hold...",Standard-balls,Catches a wild Pokémon every time.
1,2,Ultra-ball,"[countable, consumable, usable-in-battle, hold...",Standard-balls,Tries to catch a wild Pokémon. Success rate i...
2,3,Great-ball,"[countable, consumable, usable-in-battle, hold...",Standard-balls,Tries to catch a wild Pokémon. Success rate i...
3,4,Poke-ball,"[countable, consumable, usable-in-battle, hold...",Standard-balls,Tries to catch a wild Pokémon.
4,5,Safari-ball,"[countable, consumable, usable-in-battle, hold...",Standard-balls,Tries to catch a wild Pokémon in the Great Mar...


In [8]:
#Tolgo tutti i - e impongo che tutte le parole abbiano la prima lettera maisucola, in modo tale da facilitare il matching

#Dataset API
item_api['Name'] = item_api['Name'].str.replace('-',' ')
item_api['Name'] = item_api['Name'].str.title()

item_api['Category'] = item_api['Category'].str.replace('-',' ')
item_api['Category'] = item_api['Category'].str.title()

item_api.sort_values('Name', inplace=True)
item_api.reset_index(inplace=True)

item_api.drop(['index', 'ID'], axis=1, inplace=True)
item_api.head()

,Name,Attributes,Category,Effect
0,Ability Capsule,[],Vitamins,Switches a Pokémon between its two possible (n...
1,Ability Patch,[],Vitamins,None
2,Ability Urge,[],Miracle Shooter,Forcibly activates a friendly Pokémon's ability.
3,Abomasite,[],Mega Stones,Held: Allows Abomasnow to Mega Evolve into Meg...
4,Abra Candy,[],Species Candies,None


In [9]:
#Dataset scraping
item_scrap['Name'] = item_scrap['Name'].str.replace('-',' ')
item_scrap['Name'] = item_scrap['Name'].str.title()
item_scrap.head()

,Name,Category,Effect,sprite_url
0,Ability Capsule,Hold items,A capsule that allows a Pokémon with two Abili...,https://img.pokemondb.net/sprites/items/abilit...
1,Ability Patch,General items,Changes a Pokémon's ability to its Hidden Abil...,https://img.pokemondb.net/s.png
2,Ability Urge,Battle items,"When used, it activates the Ability of an ally...",https://img.pokemondb.net/sprites/items/abilit...
3,Abomasite,Hold items,Enables Abomasnow to Mega Evolve during battle.,https://img.pokemondb.net/sprites/items/abomas...
4,Absolite,Hold items,Enables Absol to Mega Evolve during battle.,https://img.pokemondb.net/sprites/items/absoli...


In [11]:
#Inizializzo l'indexer per l'integrazione e il matching
indexer = rl.Index()
indexer.full()

<Index>

In [12]:
candidates = indexer.index(item_api, item_scrap)
print(len(candidates))

1542720


In [13]:
#Voglio tenere tutti i match esatti fra gli strumenti, ma tengo anche quelli molto simili
compare = rl.Compare()
compare.string('Name', 'Name', threshold=0.9, label='Name')
compare.exact('Name', 'Name', label='Exact')

features = compare.compute(candidates, item_api, item_scrap)

In [14]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0        826
1.0         42
0.0    1541852
dtype: int64

Ci sono 826 match esatti, 42 match potenziali (stringhe di testo molto simili ma non esattamente identiche)

In [15]:
potential_matches = features[features.sum(axis=1) > 0].reset_index()
potential_matches['Score'] = potential_matches.loc[:, 'Name':'Exact'].sum(axis=1)
potential_matches.head()

,level_0,level_1,Name,Exact,Score
0,0,0,1.0,1,2.0
1,1,1,1.0,1,2.0
2,2,2,1.0,1,2.0
3,3,3,1.0,1,2.0
4,5,4,1.0,1,2.0


In [16]:
#Suddivido i match esatti e quelli simili
exact_matches = potential_matches[potential_matches['Score'] == 2.0]
temp = potential_matches[potential_matches['Score'] == 1.0]

### Tratto i match potenziali

In [17]:
#La soglia l'ho posta molto alta per non avere un numero eccessivo di match potenziali
item_similar = pd.merge(temp, item_api, left_on="level_0", right_index=True)
item_similar = pd.merge(item_similar, item_scrap, left_on="level_1", right_index=True)
item_similar.reset_index(inplace=True)
item_similar.head(50)

,index,level_0,level_1,Name_x,Exact,Score,Name_y,Attributes,Category_x,Effect_x,Name,Category_y,Effect_y,sprite_url
0,75,128,94,1.0,0,1.0,Charizardite X,[],Mega Stones,Held: Allows Charizard to Mega Evolve into Meg...,Charizardite Y,Hold items,Enables Charizard to Mega Evolve during battle.,https://img.pokemondb.net/sprites/items/chariz...
1,76,129,93,1.0,0,1.0,Charizardite Y,[],Mega Stones,Held: Allows Charizard to Mega Evolve into Meg...,Charizardite X,Hold items,Enables Charizard to Mega Evolve during battle.,https://img.pokemondb.net/sprites/items/chariz...
2,115,218,138,1.0,0,1.0,Dire Hit 2,[],Miracle Shooter,Raises critical hit rate by two stages in batt...,Dire Hit 3,Battle items,Greatly raises a Pokémon's critical-hit ratio ...,https://img.pokemondb.net/sprites/items/dire-h...
3,116,219,137,1.0,0,1.0,Dire Hit 3,[],Miracle Shooter,Raises critical hit rate by three stages in ba...,Dire Hit 2,Battle items,Raises a Pokémon's critical-hit ratio in battle.,https://img.pokemondb.net/sprites/items/dire-h...
4,153,582,179,1.0,0,1.0,Exp Candy L,[],Vitamins,None,Exp. Candy L,Medicine,NaN,https://img.pokemondb.net/s.png
5,154,583,180,1.0,0,1.0,Exp Candy M,[],Vitamins,None,Exp. Candy M,Medicine,NaN,https://img.pokemondb.net/s.png
6,155,584,181,1.0,0,1.0,Exp Candy S,[],Vitamins,None,Exp. Candy S,Medicine,NaN,https://img.pokemondb.net/s.png
7,156,585,182,1.0,0,1.0,Exp Candy Xl,[],Vitamins,None,Exp. Candy Xl,Medicine,"Increases the Pokémon's Exp points by 30,000.",https://img.pokemondb.net/s.png
8,157,586,183,1.0,0,1.0,Exp Candy Xs,[],Vitamins,None,Exp. Candy Xs,Medicine,NaN,https://img.pokemondb.net/s.png
9,158,588,184,1.0,0,1.0,Exp Share,"[holdable, holdable-active]",Training,Held: Half the experience from a battle is spl...,Exp. Share,Hold items,An item to be held by a Pokémon. The holder ge...,https://img.pokemondb.net/sprites/items/exp-sh...


Guardando i nomi di questi match potenziali si vede che quelli che effettivamente coincidono sono quelli in cui da una parte abbiamo Exp, Spec, Sp, ... senza il . finale, mentre dall'altra abbiamo Exp., Spec., Sp., ... con il . finale

Vado a tenere questi e ad eliminare gli altri:

In [20]:
item_similar['Name_y'] = item_similar['Name_y'].str.replace('Exp','Exp.')
item_similar['Name_y'] = item_similar['Name_y'].str.replace('X Sp','X Sp.')
item_similar['Name_y'] = item_similar['Name_y'].str.replace('Spec','Spec.')

In [22]:
match = item_similar[item_similar['Name_y'] == item_similar['Name']]
match.head()

,index,level_0,level_1,Name_x,Exact,Score,Name_y,Attributes,Category_x,Effect_x,Name,Category_y,Effect_y,sprite_url
4,153,582,179,1.0,0,1.0,Exp. Candy L,[],Vitamins,None,Exp. Candy L,Medicine,NaN,https://img.pokemondb.net/s.png
5,154,583,180,1.0,0,1.0,Exp. Candy M,[],Vitamins,None,Exp. Candy M,Medicine,NaN,https://img.pokemondb.net/s.png
6,155,584,181,1.0,0,1.0,Exp. Candy S,[],Vitamins,None,Exp. Candy S,Medicine,NaN,https://img.pokemondb.net/s.png
7,156,585,182,1.0,0,1.0,Exp. Candy Xl,[],Vitamins,None,Exp. Candy Xl,Medicine,"Increases the Pokémon's Exp points by 30,000.",https://img.pokemondb.net/s.png
8,157,586,183,1.0,0,1.0,Exp. Candy Xs,[],Vitamins,None,Exp. Candy Xs,Medicine,NaN,https://img.pokemondb.net/s.png


### Tratto i match esatti

In [23]:
#Aggiungo le informazioni da API (la colonna riferita a API è level_0)
item_complete = pd.merge(exact_matches, item_api, left_on="level_0", right_index=True)

#Aggiungo le informazioni da scraping (la colonna riferita a scraping è level_1)
item_complete = pd.merge(item_complete, item_scrap, left_on="level_1", right_index=True)

item_complete

,level_0,level_1,Name_x,Exact,Score,Name_y,Attributes,Category_x,Effect_x,Name,Category_y,Effect_y,sprite_url
0,0,0,1.0,1,2.0,Ability Capsule,[],Vitamins,Switches a Pokémon between its two possible (n...,Ability Capsule,Hold items,A capsule that allows a Pokémon with two Abili...,https://img.pokemondb.net/sprites/items/abilit...
1,1,1,1.0,1,2.0,Ability Patch,[],Vitamins,None,Ability Patch,General items,Changes a Pokémon's ability to its Hidden Abil...,https://img.pokemondb.net/s.png
2,2,2,1.0,1,2.0,Ability Urge,[],Miracle Shooter,Forcibly activates a friendly Pokémon's ability.,Ability Urge,Battle items,"When used, it activates the Ability of an ally...",https://img.pokemondb.net/sprites/items/abilit...
3,3,3,1.0,1,2.0,Abomasite,[],Mega Stones,Held: Allows Abomasnow to Mega Evolve into Meg...,Abomasite,Hold items,Enables Abomasnow to Mega Evolve during battle.,https://img.pokemondb.net/sprites/items/abomas...
4,5,4,1.0,1,2.0,Absolite,[],Mega Stones,Held: Allows Absol to Mega Evolve into Mega Ab...,Absolite,Hold items,Enables Absol to Mega Evolve during battle.,https://img.pokemondb.net/sprites/items/absoli...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,1597,955,1.0,1,2.0,Yellow Scarf,[holdable-active],Scarves,Raises the holder's Toughness while in a contest.,Yellow Scarf,General items,Raises holder's Tough aspect in a Contest.,https://img.pokemondb.net/sprites/items/yellow...
864,1598,956,1.0,1,2.0,Yellow Shard,[underground],Collectibles,No effect. Can be traded for items or moves.,Yellow Shard,General items,A small yellow shard. It appears to be from so...,https://img.pokemondb.net/sprites/items/yellow...
865,1601,957,1.0,1,2.0,Zap Plate,"[holdable, holdable-active, underground]",Plates,Held: Electric-Type moves from holder do 20% m...,Zap Plate,Hold items,Increases power of Electric-type moves. Change...,https://img.pokemondb.net/sprites/items/zap-pl...
866,1603,958,1.0,1,2.0,Zinc,"[countable, consumable, usable-overworld, usab...",Vitamins,Raises Special Defense and happiness.,Zinc,Medicine,Increases Special Defense EVs by 10.,https://img.pokemondb.net/sprites/items/zinc.png


### Concatenazione match esatti e simili

In [24]:
item_total = pd.concat([item_complete, match])
item_total.sort_values('Name', inplace=True)
item_total.drop(["level_0", "level_1", "Name_x", "Exact", "Score", "Name_y", "index"], axis = 1, inplace=True)
item_total.tail()

,Attributes,Category_x,Effect_x,Name,Category_y,Effect_y,sprite_url
863,[holdable-active],Scarves,Raises the holder's Toughness while in a contest.,Yellow Scarf,General items,Raises holder's Tough aspect in a Contest.,https://img.pokemondb.net/sprites/items/yellow...
864,[underground],Collectibles,No effect. Can be traded for items or moves.,Yellow Shard,General items,A small yellow shard. It appears to be from so...,https://img.pokemondb.net/sprites/items/yellow...
865,"[holdable, holdable-active, underground]",Plates,Held: Electric-Type moves from holder do 20% m...,Zap Plate,Hold items,Increases power of Electric-type moves. Change...,https://img.pokemondb.net/sprites/items/zap-pl...
866,"[countable, consumable, usable-overworld, usab...",Vitamins,Raises Special Defense and happiness.,Zinc,Medicine,Increases Special Defense EVs by 10.,https://img.pokemondb.net/sprites/items/zinc.png
867,"[holdable, holdable-active]",Held Items,Held: Provides a 1/5 (20%) boost in accuracy i...,Zoom Lens,Hold items,Raises a move's accuracy if the holder moves a...,https://img.pokemondb.net/sprites/items/zoom-l...


In [25]:
item_total.reset_index(inplace=True)
item_total.drop(["index"], axis = 1, inplace=True)
item_total.tail()

,Attributes,Category_x,Effect_x,Name,Category_y,Effect_y,sprite_url
835,[holdable-active],Scarves,Raises the holder's Toughness while in a contest.,Yellow Scarf,General items,Raises holder's Tough aspect in a Contest.,https://img.pokemondb.net/sprites/items/yellow...
836,[underground],Collectibles,No effect. Can be traded for items or moves.,Yellow Shard,General items,A small yellow shard. It appears to be from so...,https://img.pokemondb.net/sprites/items/yellow...
837,"[holdable, holdable-active, underground]",Plates,Held: Electric-Type moves from holder do 20% m...,Zap Plate,Hold items,Increases power of Electric-type moves. Change...,https://img.pokemondb.net/sprites/items/zap-pl...
838,"[countable, consumable, usable-overworld, usab...",Vitamins,Raises Special Defense and happiness.,Zinc,Medicine,Increases Special Defense EVs by 10.,https://img.pokemondb.net/sprites/items/zinc.png
839,"[holdable, holdable-active]",Held Items,Held: Provides a 1/5 (20%) boost in accuracy i...,Zoom Lens,Hold items,Raises a move's accuracy if the holder moves a...,https://img.pokemondb.net/sprites/items/zoom-l...


In [26]:
item_total = item_total.reindex(columns=['Name', 'Effect_x', 'Effect_y', 'Attributes', 'Category_x', 'Category_y', 
                                         'sprite_url'])

item_total.rename(columns={"Name": "Name", "Effect_x": "Effect_API", "Effect_y": "Effect_SCRAPING", "Attributes": "Attributes",
                          "Category_x": "Category_API", "Category_y": "Category_SCRAPING", "sprite_url": "sprite_url"})

,Name,Effect_API,Effect_SCRAPING,Attributes,Category_API,Category_SCRAPING,sprite_url
0,Ability Capsule,Switches a Pokémon between its two possible (n...,A capsule that allows a Pokémon with two Abili...,[],Vitamins,Hold items,https://img.pokemondb.net/sprites/items/abilit...
1,Ability Patch,None,Changes a Pokémon's ability to its Hidden Abil...,[],Vitamins,General items,https://img.pokemondb.net/s.png
2,Ability Urge,Forcibly activates a friendly Pokémon's ability.,"When used, it activates the Ability of an ally...",[],Miracle Shooter,Battle items,https://img.pokemondb.net/sprites/items/abilit...
3,Abomasite,Held: Allows Abomasnow to Mega Evolve into Meg...,Enables Abomasnow to Mega Evolve during battle.,[],Mega Stones,Hold items,https://img.pokemondb.net/sprites/items/abomas...
4,Absolite,Held: Allows Absol to Mega Evolve into Mega Ab...,Enables Absol to Mega Evolve during battle.,[],Mega Stones,Hold items,https://img.pokemondb.net/sprites/items/absoli...
...,...,...,...,...,...,...,...
835,Yellow Scarf,Raises the holder's Toughness while in a contest.,Raises holder's Tough aspect in a Contest.,[holdable-active],Scarves,General items,https://img.pokemondb.net/sprites/items/yellow...
836,Yellow Shard,No effect. Can be traded for items or moves.,A small yellow shard. It appears to be from so...,[underground],Collectibles,General items,https://img.pokemondb.net/sprites/items/yellow...
837,Zap Plate,Held: Electric-Type moves from holder do 20% m...,Increases power of Electric-type moves. Change...,"[holdable, holdable-active, underground]",Plates,Hold items,https://img.pokemondb.net/sprites/items/zap-pl...
838,Zinc,Raises Special Defense and happiness.,Increases Special Defense EVs by 10.,"[countable, consumable, usable-overworld, usab...",Vitamins,Medicine,https://img.pokemondb.net/sprites/items/zinc.png
